In [1]:
import os 
import pandas as pd 
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd
import fastparquet
from sklearn.svm import OneClassSVM
try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")
    
train = pd.read_parquet('train.parquet.gzip')
test = pd.read_parquet('test.parquet.gzip')

In [3]:
from sklearn.utils import class_weight

classes = np.unique(train.FlagImpaye)
cw = class_weight.compute_class_weight(class_weight = 'balanced',classes =  np.unique(train.FlagImpaye),y= train.FlagImpaye)
weights = dict(zip(classes,cw))

In [4]:
Xtrain = train.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction", "Unnamed: 0"], axis = 1)

Ytrain = pd.DataFrame(train.FlagImpaye)
Ytrain = Ytrain['FlagImpaye'].astype('int')

Xtest  = test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction", 'Unnamed: 0'], axis = 1)
Ytest  = test.FlagImpaye

In [5]:
from xgboost import XGBRFClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
model = RandomForestClassifier(
                        class_weight =weights)
model.fit(Xtrain, Ytrain)
y_score = model.predict_proba(Xtest)
y_pred = model.predict(Xtest)
from sklearn.metrics import classification_report
tab = classification_report(Ytest, y_pred)
print(tab)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    740838
           1       1.00      0.65      0.79      6573

    accuracy                           1.00    747411
   macro avg       1.00      0.82      0.89    747411
weighted avg       1.00      1.00      1.00    747411



In [9]:
tab = classification_report(Ytest, y_pred,output_dict=True)
tab = pd.DataFrame(tab).transpose()
tab.to_csv("C:/Users/Sam/Documents/GitHub/FouilleDonneeMassive/RandomForest/Tab_POidsBagging.csv")

In [6]:
from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(base_estimator=model,
                                random_state=0)
bagging_clf.fit(Xtrain, Ytrain)
y_score = bagging_clf.predict_proba(Xtest)
y_pred2 = bagging_clf.predict(Xtest)
tab2 = classification_report(Ytest, y_pred2)
print(tab2)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    740838
           1       1.00      0.65      0.79      6573

    accuracy                           1.00    747411
   macro avg       1.00      0.82      0.89    747411
weighted avg       1.00      1.00      1.00    747411



In [ ]:
from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(base_estimator=XGBRFClassifier(max_depth=6,gamma = 1, learning_rate =  0.05),
                                random_state=0)
bagging_clf.fit(Xtrain, Ytrain)
y_score = bagging_clf.predict_proba(Xtest)
y_pred3 = bagging_clf.predict(Xtest)
tab3 = classification_report(Ytest, y_pred3)
print(tab2)